In [0]:
spark

In [0]:
%sql
use hive_metastore.default

In [0]:
%sql
select * from books

In [0]:
spark.readStream.table('books').createOrReplaceTempView('books_streaming_temp_vw')

In [0]:
%sql
select * from books_streaming_temp_vw

In [0]:
%sql
select author,count(book_id) as book_cnt from books_streaming_temp_vw
group by author


In [0]:
%sql
select distinct author from books_streaming_temp_vw order by 1 asc

In [0]:
%sql

create or replace temp view author_book_cnt_vw as (
  select author,count(book_id) as book_cnt from books_streaming_temp_vw
group by author
)

In [0]:
spark.table('author_book_cnt_vw').writeStream.trigger(processingTime='4 seconds').outputMode('complete').option('checkpointLocation', 'dbfs:/mnt/demo/author_counts_checkpoint').table('author_counts')




In [0]:
%sql
select * from author_counts where author = 'SRP'

In [0]:
%sql
insert into books values ('B111','intro','Chris Bernhardt','Computer Science',25),
('B1111','life','SRP','avg',2500)



In [0]:
%sql
insert into books values 
('B222','end','SRP','aero',2500)


In [0]:
spark.table('author_book_cnt_vw').writeStream.trigger(availableNow=True).outputMode('complete').option('checkpointLocation', 'dbfs:/mnt/demo/author_counts_checkpoint_2').table('author_counts').awaitTermination()



In [0]:
spark.sql("select * from author_counts where author='SRP'").show()

### **auto loader adn copy into**

In [0]:
display(dbutils.fs.ls('dbfs:/mnt/demo-datasets/bookstore/orders-raw'))

In [0]:
(
    spark.readStream.format('cloudFiles').option('cloudFiles.format','parquet').option('cloudFiles.schemaLocation','dbfs:/mnt/demo/bookstore/auto_loader_chechpoint').load('dbfs:/mnt/demo-datasets/bookstore/orders-raw').writeStream.option('checkpointLocation','dbfs:/mnt/demo/bookstore/auto_loader_chechpoint_2').table('orders_updates')
)


In [0]:
%sql
select * from orders_updates

In [0]:
%sql
select count(*) as cnt from orders_updates

In [0]:
%run /Workspace/Users/sandippatelaero@gmail.com/Databricks-Certified-Data-Engineer-Associate/Includes/Copy-Datasets

In [0]:
load_new_data()


In [0]:
display(dbutils.fs.ls('dbfs:/mnt/demo-datasets/bookstore/orders-raw'))

In [0]:
%sql
describe history orders_updates

In [0]:
%sql
drop table orders_updates

In [0]:

display(dbutils.fs.ls('dbfs:/mnt/demo/bookstore/auto_loader_chechpoint/_schemas/0'))
